In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import geopandas as gp

In [ ]:
# Set file path
csvpath = "Resources/2016-precinct-president.csv"

# Read csv and preview dataframe
prec_df = pd.read_csv(csvpath, encoding="ISO-8859-1")
pd.set_option('display.max_columns', None)
prec_df.head()

# Exploring the data

In [ ]:
# List of Columns
prec_df.columns

In [ ]:
# Getting the value counts for all the states
prec_df['state'].value_counts()

# Minnesota

In [ ]:
# Only interested in Minnesota precincts
mn_df = prec_df.loc[prec_df['state'] == 'Minnesota']
mn_df.head()

In [ ]:
# Keep only the columns we're interested in
# Want to come back and remove more once I get a little deeper into it
mn_df = mn_df[['year', 'state', 'state_postal', 'state_fips', 'state_icpsr', 'county_name',
               'county_fips', 'county_ansi', 'county_lat', 'county_long', 'jurisdiction',
               'precinct', 'candidate_normalized', 'mode', 'votes', 'candidate_fec']]
mn_df.head(11)

In [ ]:
# Create a new column for Total Votes and add that to the dataframe

# Create the column and give it an initial value of zero
mn_df['total_votes'] = 0

# Add the new column into the dataframe
mn_df = mn_df[['year', 'state', 'state_postal', 'state_fips', 'state_icpsr', 'county_name',
               'county_fips', 'county_ansi', 'county_lat', 'county_long', 'jurisdiction',
               'precinct', 'candidate_normalized', 'mode', 'votes', 'total_votes', 'candidate_fec']]

# Preview
mn_df.head()

In [ ]:
# Groupby the precincts
mn_groupby = mn_df.groupby(['precinct']).sum()
mn_groupby

In [ ]:
# Merge the two dataframes, mn_groupby and mn_df
merged_df = mn_df.merge(mn_groupby, on = "precinct")

# Preview
merged_df

In [ ]:
# Pull out the desired columns
merged_df = merged_df[['year_x', 'state', 'state_postal', 'county_name', 'county_fips_x', 'county_ansi_x',
                       'county_lat_x', 'county_long_x', 'jurisdiction', 'precinct', 'candidate_normalized', 
                       'votes_x', 'votes_y']]
merged_df

In [ ]:
# Rename the columns
merged_df = merged_df.rename(columns={'year_x': 'year', 'county_fips_x': 'county_fips',
                                      'county_ansi_x': 'county_ansi', 'county_lat_x': 'county_lat',
                                      'county_long_x': 'county_long', 'candidate_normalized': 'candidate',
                                      'votes_x': 'votes', 'votes_y': 'total_votes'})
merged_df.head()

In [ ]:
# Add a percent column which will determine how much percent of the precinct's vote each candidate got
merged_df['percent'] = (merged_df['votes'] / merged_df['total_votes']) * 100
merged_df.head()

In [ ]:
#merged_df[merged_df.county_fips == 27001.0]
merged_df[merged_df.precinct == 'BALL BLUFF TWP|15']

In [ ]:
merged_df[merged_df.county_name == 'Aitkin County']

# GeoPandas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gp
from geopandas import GeoDataFrame

In [56]:
# load shp file extracted from zipfile
prec_shp = gp.read_file("Resources/bdry_votingdistricts.shp" )
prec_shp.head()

,VTDID,PCTNAME,PCTCODE,SHORTLABEL,MCDNAME,MCDCODE,MCDFIPS,MCDGNIS,CTU_TYPE,COUNTYNAME,COUNTYCODE,COUNTYFIPS,CONGDIST,MNSENDIST,MNLEGDIST,CTYCOMDIST,JUDDIST,SWCDIST,SWCDIST_N,WARD,HOSPDIST,HOSPDIST_N,PARKDIST,PARKDIST_N,geometry
0,271730045,Friendship Twp,0045,None,Friendship Twp,035,22832,00664234,township,Yellow Medicine,87,173,7,16,16A,3,08,5173,Yellow Medicine,None,None,None,None,None,"POLYGON ((284369.269 4964918.229, 284321.301 4..."
1,270910110,Galena Twp,0110,None,Galena Twp,055,23030,00664240,township,Martin,46,091,1,23,23A,5,05,6091,Martin,None,None,None,None,None,"POLYGON ((362977.491 4856419.851, 362974.359 4..."
2,270930045,Darwin Twp,0045,None,Darwin Twp,040,14860,00663929,township,Meeker,47,093,7,18,18A,1,08,2093,Meeker,None,None,None,None,None,"POLYGON ((391605.300 5000799.731, 391581.799 4..."
3,271370060,Biwabik Twp,0060,None,Biwabik Twp,060,06166,00663605,township,St. Louis,69,137,8,6,6B,4,06,3137,North St. Louis,None,None,None,None,None,"MULTIPOLYGON (((544403.320 5260705.800, 544214..."
4,270010015,Ball Bluff Twp,0015,None,Ball Bluff Twp,015,03358,00663498,township,Aitkin,1,001,8,10,10B,5,09,3001,Aitkin,None,None,None,None,None,"POLYGON ((485475.521 5207980.031, 485475.766 5..."


In [ ]:
prec_shp[prec_shp.VTDID == '270010005'].head()

In [ ]:
ball_bluff = prec_shp[prec_shp.PCTNAME == 'Ball Bluff Twp']
ball_bluff
#'''MCDGNIS (column from prec_shp dataframe) is close to merged_df[county_ansi], but not exact'''

In [ ]:
print(ball_bluff['geometry'])

In [57]:
# Rename the columns that I'm going to keep
prec_shp = prec_shp.rename(columns = {'PCTNAME': 'precinct',
                                      'COUNTYFIPS': 'county_fips',
                                      'VTIDID':'VTIDID'})
prec_shp.head()

,VTDID,precinct,PCTCODE,SHORTLABEL,MCDNAME,MCDCODE,MCDFIPS,MCDGNIS,CTU_TYPE,COUNTYNAME,COUNTYCODE,county_fips,CONGDIST,MNSENDIST,MNLEGDIST,CTYCOMDIST,JUDDIST,SWCDIST,SWCDIST_N,WARD,HOSPDIST,HOSPDIST_N,PARKDIST,PARKDIST_N,geometry
0,271730045,Friendship Twp,0045,None,Friendship Twp,035,22832,00664234,township,Yellow Medicine,87,173,7,16,16A,3,08,5173,Yellow Medicine,None,None,None,None,None,"POLYGON ((284369.269 4964918.229, 284321.301 4..."
1,270910110,Galena Twp,0110,None,Galena Twp,055,23030,00664240,township,Martin,46,091,1,23,23A,5,05,6091,Martin,None,None,None,None,None,"POLYGON ((362977.491 4856419.851, 362974.359 4..."
2,270930045,Darwin Twp,0045,None,Darwin Twp,040,14860,00663929,township,Meeker,47,093,7,18,18A,1,08,2093,Meeker,None,None,None,None,None,"POLYGON ((391605.300 5000799.731, 391581.799 4..."
3,271370060,Biwabik Twp,0060,None,Biwabik Twp,060,06166,00663605,township,St. Louis,69,137,8,6,6B,4,06,3137,North St. Louis,None,None,None,None,None,"MULTIPOLYGON (((544403.320 5260705.800, 544214..."
4,270010015,Ball Bluff Twp,0015,None,Ball Bluff Twp,015,03358,00663498,township,Aitkin,1,001,8,10,10B,5,09,3001,Aitkin,None,None,None,None,None,"POLYGON ((485475.521 5207980.031, 485475.766 5..."


In [ ]:
# Need to merge merged_df and prec_shp dataframes


In [58]:
# Rename the columns that I'm going to keep
prec_shp = prec_shp.rename(columns = {'PCTNAME': 'precinct',
                                      'COUNTYFIPS': 'county_fips',
                                      'VTDID':'VTDID'})
prec_shp.head()

,VTDID,precinct,PCTCODE,SHORTLABEL,MCDNAME,MCDCODE,MCDFIPS,MCDGNIS,CTU_TYPE,COUNTYNAME,COUNTYCODE,county_fips,CONGDIST,MNSENDIST,MNLEGDIST,CTYCOMDIST,JUDDIST,SWCDIST,SWCDIST_N,WARD,HOSPDIST,HOSPDIST_N,PARKDIST,PARKDIST_N,geometry
0,271730045,Friendship Twp,0045,None,Friendship Twp,035,22832,00664234,township,Yellow Medicine,87,173,7,16,16A,3,08,5173,Yellow Medicine,None,None,None,None,None,"POLYGON ((284369.269 4964918.229, 284321.301 4..."
1,270910110,Galena Twp,0110,None,Galena Twp,055,23030,00664240,township,Martin,46,091,1,23,23A,5,05,6091,Martin,None,None,None,None,None,"POLYGON ((362977.491 4856419.851, 362974.359 4..."
2,270930045,Darwin Twp,0045,None,Darwin Twp,040,14860,00663929,township,Meeker,47,093,7,18,18A,1,08,2093,Meeker,None,None,None,None,None,"POLYGON ((391605.300 5000799.731, 391581.799 4..."
3,271370060,Biwabik Twp,0060,None,Biwabik Twp,060,06166,00663605,township,St. Louis,69,137,8,6,6B,4,06,3137,North St. Louis,None,None,None,None,None,"MULTIPOLYGON (((544403.320 5260705.800, 544214..."
4,270010015,Ball Bluff Twp,0015,None,Ball Bluff Twp,015,03358,00663498,township,Aitkin,1,001,8,10,10B,5,09,3001,Aitkin,None,None,None,None,None,"POLYGON ((485475.521 5207980.031, 485475.766 5..."


In [59]:
prec_shp = prec_shp[['VTDID','precinct', 'county_fips', 'geometry']]
prec_shp

,VTDID,precinct,county_fips,geometry
0,271730045,Friendship Twp,173,"POLYGON ((284369.269 4964918.229, 284321.301 4..."
1,270910110,Galena Twp,091,"POLYGON ((362977.491 4856419.851, 362974.359 4..."
2,270930045,Darwin Twp,093,"POLYGON ((391605.300 5000799.731, 391581.799 4..."
3,271370060,Biwabik Twp,137,"MULTIPOLYGON (((544403.320 5260705.800, 544214..."
4,270010015,Ball Bluff Twp,001,"POLYGON ((485475.521 5207980.031, 485475.766 5..."
...,...,...,...,...
4105,270950015,Borgholm Twp,095,"MULTIPOLYGON (((450879.660 5068471.679, 451005..."
4106,270530505,Corcoran P-2,053,"POLYGON ((455732.119 5000074.516, 457351.820 5..."
4107,271710005,Albertville P-1,171,"POLYGON ((447828.738 5007699.654, 447826.627 5..."
4108,271710127,Monticello P-2,171,"POLYGON ((439564.488 5015565.645, 439569.764 5..."


In [40]:
prec_shp[prec_shp.county_fips == '001' ]

,precinct,county_fips,geometry


In [ ]:
# prec_shp['county_fips'] = '27' + prec_shp['county_fips']

In [60]:
prec_shp['county_fips'] = prec_shp['county_fips'].astype(float)
prec_shp.head(1)

,VTDID,precinct,county_fips,geometry
0,271730045,Friendship Twp,173.0,"POLYGON ((284369.269 4964918.229, 284321.301 4..."


In [45]:
shp_merge = prec_shp.merge(merged_df, on='county_fips')
shp_merge

,precinct_x,county_fips,geometry,year,state,state_postal,county_name,county_ansi,county_lat,county_long,jurisdiction,precinct_y,candidate,votes,total_votes,percent


In [25]:
shp_merge[shp_merge.county_fips == 27077.0]


,precinct_x,county_fips,geometry,year,state,state_postal,county_name,county_ansi,county_lat,county_long,jurisdiction,precinct_y,candidate,votes,total_votes,percent


# Comapring merged_df and prec_shp


In [ ]:
merged_df

In [61]:
# Set file path
csvpath = "Resources/2016_precincts_newVersion.csv"

# Read csv and preview dataframe
prec_df = pd.read_csv(csvpath, encoding="ISO-8859-1")
pd.set_option('display.max_columns', None)
prec_df.head()

,VTDID,PCTNAME,PCTCODE,MCDNAME,COUNTYNAME,COUNTYCODE,CONGDIST,MNSENDIST,MNLEGDIST,CTYCOMDIST,JUDDIST,SWCDIST,WARD,HOSPDIST,PARKDIST,TABSYSTEM,TABMODEL,MAILBALLOT,REG7AM,EDR,SIGNATURES,AB_MB,FEDONLYAB,PRESONLYAB,TOTVOTING,USPRSR,USPRSDFL,USPRSCP,USPRSLMN,USPRSSWP,USPRSGP,USPRSADP,USPRSIP,USPRSLIB,USPRSWI,USPRSTOTAL,USREPR,USREPDFL,USREPWI,USREPTOTAL,MNSENR,MNSENDFL,MNSENWI,MNSENTOTAL,MNLEGR,MNLEGDFL,MNLEGWI,MNLEGTOTAL,MNCA1YES,MNCA1NO,MNCA1EST,MNCA1TOTAL
0,270010005.0,Aitkin,5.0,Aitkin,Aitkin,1.0,8.0,10.0,10B,1.0,9.0,3001.0,NaN,NaN,NaN,Precinct Tabulator,ES&S Model 100,NO,1141,110,758,242,0,0,1000,552,358,3,7,1,2,0,21,36,6,986,491,470,0,961,574,368,1,943,561,411,0,972,731,224,45,1000
1,270010010.0,Aitkin Twp,10.0,Aitkin Twp,Aitkin,1.0,8.0,10.0,10B,1.0,9.0,3001.0,NaN,NaN,NaN,Precinct Tabulator,ES&S Model 100,NO,588,44,437,113,0,0,550,356,154,1,2,0,5,0,7,18,4,547,332,206,0,538,381,150,0,531,370,173,0,543,438,95,17,550
2,270010015.0,Ball Bluff Twp,15.0,Ball Bluff Twp,Aitkin,1.0,8.0,10.0,10B,5.0,9.0,3001.0,NaN,NaN,NaN,Precinct Tabulator,ES&S Model 100,NO,179,18,145,19,0,0,164,96,57,0,1,0,1,0,2,6,1,164,80,82,0,162,83,77,0,160,79,79,0,158,131,32,1,164
3,270010020.0,Balsam Twp,20.0,Balsam Twp,Aitkin,1.0,8.0,10.0,10B,5.0,9.0,3001.0,NaN,NaN,NaN,Central Count,ES&S Model 100,YES,22,0,0,19,0,0,19,9,8,0,0,0,0,0,0,0,1,18,9,10,0,19,10,9,0,19,8,11,0,19,15,2,2,19
4,270010025.0,Beaver Twp,25.0,Beaver Twp,Aitkin,1.0,8.0,10.0,10B,3.0,9.0,3001.0,NaN,NaN,NaN,Central Count,ES&S Model 100,YES,42,0,0,37,0,0,37,16,19,0,2,0,0,0,0,0,0,37,11,25,0,36,13,20,0,33,11,23,0,34,31,6,0,37


In [62]:
prec_df = prec_df[['VTDID', 'PCTNAME', 'PCTCODE', 'MCDNAME', 'COUNTYNAME', 'COUNTYCODE',
                   'USPRSR', 'USPRSDFL', 'USPRSTOTAL']][:-1]
prec_df

,VTDID,PCTNAME,PCTCODE,MCDNAME,COUNTYNAME,COUNTYCODE,USPRSR,USPRSDFL,USPRSTOTAL
0,270010005.0,Aitkin,5.0,Aitkin,Aitkin,1.0,552,358,986
1,270010010.0,Aitkin Twp,10.0,Aitkin Twp,Aitkin,1.0,356,154,547
2,270010015.0,Ball Bluff Twp,15.0,Ball Bluff Twp,Aitkin,1.0,96,57,164
3,270010020.0,Balsam Twp,20.0,Balsam Twp,Aitkin,1.0,9,8,18
4,270010025.0,Beaver Twp,25.0,Beaver Twp,Aitkin,1.0,16,19,37
...,...,...,...,...,...,...,...,...,...
4115,271730145.0,Swede Prairie Twp,145.0,Swede Prairie Twp,Yellow Medicine,87.0,44,13,61
4116,271730150.0,Tyro Twp,150.0,Tyro Twp,Yellow Medicine,87.0,77,19,97
4117,271730155.0,Wergeland Twp,155.0,Wergeland Twp,Yellow Medicine,87.0,81,24,110
4118,271730160.0,Wood Lake,160.0,Wood Lake,Yellow Medicine,87.0,138,51,209


In [63]:
prec_df = prec_df.rename(columns = {'MCDNAME': 'Munic/Unorg Terr Name',
                                    'USPRSR': 'Trump', 'USPRSDFL': 'Clinton',
                                    'USPRSTOTAL': 'Total Votes'})
prec_df.head()

,VTDID,PCTNAME,PCTCODE,Munic/Unorg Terr Name,COUNTYNAME,COUNTYCODE,Trump,Clinton,Total Votes
0,270010005.0,Aitkin,5.0,Aitkin,Aitkin,1.0,552,358,986
1,270010010.0,Aitkin Twp,10.0,Aitkin Twp,Aitkin,1.0,356,154,547
2,270010015.0,Ball Bluff Twp,15.0,Ball Bluff Twp,Aitkin,1.0,96,57,164
3,270010020.0,Balsam Twp,20.0,Balsam Twp,Aitkin,1.0,9,8,18
4,270010025.0,Beaver Twp,25.0,Beaver Twp,Aitkin,1.0,16,19,37


In [64]:
prec_df['Trump %'] = ((prec_df['Trump'] / prec_df['Total Votes']) * 100).round(2)
prec_df['Clinton %'] = ((prec_df['Clinton'] / prec_df['Total Votes']) * 100).round(2)
prec_df.head()

,VTDID,PCTNAME,PCTCODE,Munic/Unorg Terr Name,COUNTYNAME,COUNTYCODE,Trump,Clinton,Total Votes,Trump %,Clinton %
0,270010005.0,Aitkin,5.0,Aitkin,Aitkin,1.0,552,358,986,55.98,36.31
1,270010010.0,Aitkin Twp,10.0,Aitkin Twp,Aitkin,1.0,356,154,547,65.08,28.15
2,270010015.0,Ball Bluff Twp,15.0,Ball Bluff Twp,Aitkin,1.0,96,57,164,58.54,34.76
3,270010020.0,Balsam Twp,20.0,Balsam Twp,Aitkin,1.0,9,8,18,50.00,44.44
4,270010025.0,Beaver Twp,25.0,Beaver Twp,Aitkin,1.0,16,19,37,43.24,51.35


In [68]:
prec_shp.head()
prec_shp.dtypes

VTDID            object
precinct         object
county_fips     float64
geometry       geometry
dtype: object

In [67]:
prec_df.head()
prec_df.dtypes

VTDID                    float64
PCTNAME                   object
PCTCODE                  float64
Munic/Unorg Terr Name     object
COUNTYNAME                object
COUNTYCODE               float64
Trump                      int64
Clinton                    int64
Total Votes                int64
Trump %                  float64
Clinton %                float64
dtype: object

In [70]:
prec_shp['VTDID'] = prec_shp['VTDID'].astype(float)
prec_shp

,VTDID,precinct,county_fips,geometry
0,271730045.0,Friendship Twp,173.0,"POLYGON ((284369.269 4964918.229, 284321.301 4..."
1,270910110.0,Galena Twp,91.0,"POLYGON ((362977.491 4856419.851, 362974.359 4..."
2,270930045.0,Darwin Twp,93.0,"POLYGON ((391605.300 5000799.731, 391581.799 4..."
3,271370060.0,Biwabik Twp,137.0,"MULTIPOLYGON (((544403.320 5260705.800, 544214..."
4,270010015.0,Ball Bluff Twp,1.0,"POLYGON ((485475.521 5207980.031, 485475.766 5..."
...,...,...,...,...
4105,270950015.0,Borgholm Twp,95.0,"MULTIPOLYGON (((450879.660 5068471.679, 451005..."
4106,270530505.0,Corcoran P-2,53.0,"POLYGON ((455732.119 5000074.516, 457351.820 5..."
4107,271710005.0,Albertville P-1,171.0,"POLYGON ((447828.738 5007699.654, 447826.627 5..."
4108,271710127.0,Monticello P-2,171.0,"POLYGON ((439564.488 5015565.645, 439569.764 5..."


In [71]:
merged_df = pd.merge(prec_shp, prec_df, on = 'VTDID')

In [72]:
merged_df

,VTDID,precinct,county_fips,geometry,PCTNAME,PCTCODE,Munic/Unorg Terr Name,COUNTYNAME,COUNTYCODE,Trump,Clinton,Total Votes,Trump %,Clinton %
0,271730045.0,Friendship Twp,173.0,"POLYGON ((284369.269 4964918.229, 284321.301 4...",Friendship Twp,45.0,Friendship Twp,Yellow Medicine,87.0,91,33,133,68.42,24.81
1,270910110.0,Galena Twp,91.0,"POLYGON ((362977.491 4856419.851, 362974.359 4...",Galena Twp.,110.0,Galena Twp,Martin,46.0,84,43,130,64.62,33.08
2,270930045.0,Darwin Twp,93.0,"POLYGON ((391605.300 5000799.731, 391581.799 4...",Darwin Twp.,45.0,Darwin Twp,Meeker,47.0,295,97,425,69.41,22.82
3,271370060.0,Biwabik Twp,137.0,"MULTIPOLYGON (((544403.320 5260705.800, 544214...",Biwabik Twp.,60.0,Biwabik Twp,St. Louis,69.0,289,234,556,51.98,42.09
4,270010015.0,Ball Bluff Twp,1.0,"POLYGON ((485475.521 5207980.031, 485475.766 5...",Ball Bluff Twp,15.0,Ball Bluff Twp,Aitkin,1.0,96,57,164,58.54,34.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,270531810.0,Minneapolis W-10 P-2,53.0,"POLYGON ((476878.092 4978054.295, 476875.969 4...",Minneapolis W-10 P-02,1810.0,Minneapolis,Hennepin,27.0,429,2478,3208,13.37,77.24
4097,270950015.0,Borgholm Twp,95.0,"MULTIPOLYGON (((450879.660 5068471.679, 451005...",Borgholm Twp.,15.0,Borgholm Twp,Mille Lacs,48.0,662,181,900,73.56,20.11
4098,270530505.0,Corcoran P-2,53.0,"POLYGON ((455732.119 5000074.516, 457351.820 5...",Corcoran P-02,505.0,Corcoran,Hennepin,27.0,889,454,1452,61.23,31.27
4099,271710005.0,Albertville P-1,171.0,"POLYGON ((447828.738 5007699.654, 447826.627 5...",Albertville,5.0,Albertville,Wright,86.0,2251,1110,3734,60.28,29.73


In [39]:
test = pd.DataFrame(prec_shp['VTDID'])
test

KeyError: 'VTDID'

In [ ]:
test2 = pd.DataFrame(prec_df['VTDID'])
test2

In [ ]:
test = pd.Series(prec_shp['VTDID'])
test

In [ ]:
test2 = pd.Series(prec_df['VTDID'])
test2

In [ ]:
test.compare(test2)